In [75]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.impute import SimpleImputer
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [76]:
Dataset_train=pd.read_csv('../input/janatahack-customer-segmentation/Train.csv')
Dataset=pd.read_csv('../input/janatahack-customer-segmentation/Test.csv')

Dataset=pd.concat([Dataset_train, Dataset], axis=0, ignore_index=True, sort=False)

In [77]:
Dataset.shape , Dataset_train.shape

((10695, 11), (8068, 11))

In [78]:
Dataset.isnull().sum()


ID                    0
Gender                0
Ever_Married        190
Age                   0
Graduated           102
Profession          162
Work_Experience    1098
Spending_Score        0
Family_Size         448
Var_1               108
Segmentation       2627
dtype: int64

In [79]:
Dataset['Ever_Married'].value_counts()
Dataset['Ever_Married'].fillna('NA', inplace=True)

In [80]:
Dataset['Graduated'].value_counts()
Dataset['Graduated'].fillna('NA', inplace=True)

In [81]:
Dataset['Profession'].value_counts()
Dataset['Profession'].fillna('NA', inplace=True)

In [82]:

Dataset['Work_Experience'].fillna(-1.0, inplace=True)
Dataset['Work_Experience'].value_counts()

 1.0     3127
 0.0     3087
-1.0     1098
 9.0      613
 8.0      612
 2.0      373
 4.0      346
 3.0      337
 5.0      270
 6.0      265
 7.0      256
 14.0      66
 11.0      64
 10.0      64
 12.0      60
 13.0      57
Name: Work_Experience, dtype: int64

In [83]:

Dataset['Family_Size'].fillna(-1, inplace=True)
Dataset['Family_Size'].value_counts()

 2.0    3158
 1.0    1965
 3.0    1952
 4.0    1823
 5.0     812
-1.0     448
 6.0     290
 7.0     122
 8.0      65
 9.0      60
Name: Family_Size, dtype: int64

In [84]:
Dataset['Var_1'].fillna('Unknown', inplace=True)
Dataset['Var_1'].value_counts()

Cat_6      6910
Cat_4      1475
Cat_3      1089
Cat_2       563
Cat_7       269
Cat_1       167
Cat_5       114
Unknown     108
Name: Var_1, dtype: int64

In [85]:
# Dataset['Loan_Amount_uested']=Dataset['Loan_Amount_Requested'].str.replace("," ,'')

# Dataset['Loan_Amount_Requested']=Dataset['Loan_Amount_Requested'].astype(int)

# Dataset['Loan_Amount_Requested'].describe()

# Dataset['Loan_Amount_Requested']=np.log(Dataset['Loan_Amount_Requested'])

# Dataset['Months_Since_Deliquency'].describe()

# Dataset['Months_Since_Deliquency']=np.log(Dataset['Months_Since_Deliquency'], where = Dataset['Months_Since_Deliquency']!=0)

# Dataset['Annual_Income'].describe()

# Dataset['Annual_Income']=np.log(Dataset['Annual_Income'])

# Dataset['Number_Open_Accounts'].describe()

# Dataset['Number_Open_Accounts']=np.log(Dataset['Number_Open_Accounts'], where = Dataset['Number_Open_Accounts']!=0)


# Dataset['Total_Accounts']=np.log(Dataset['Total_Accounts'])

# Dataset['Home_Owner'].fillna('unknown', inplace=True)

# Dataset['Length_Employed']=Dataset['Length_Employed'].str.replace('years', '')

# Dataset['Length_Employed']=Dataset['Length_Employed'].replace('1 year', 1)

# Dataset['Length_Employed']=Dataset['Length_Employed'].replace('10+ ', 11)

# Dataset['Length_Employed']=Dataset['Length_Employed'].replace('< 1 year', 0)

# Dataset['Length_Employed'].fillna(21, inplace=True)

# Dataset['Length_Employed']=Dataset['Length_Employed'].astype(int)

# label_encode=LabelEncoder()

# Dataset['Gender']=label_encode.fit_transform(Dataset['Gender'])

# Dataset['Purpose_Of_Loan'].value_counts()

# Dataset['Purpose_Of_Loan']=Dataset['Purpose_Of_Loan'].map({'debt_consolidation':1, 'credit_card':2,
#                                                            'home_improvement':3, 'other':4,
#                                                            'major_purchase':5, 'small_business':6,
#                                                            'car':7, 'medical':8, 
#                                                            'moving':9, 'vacation':10,
#                                                            'wedding':11, 'house':12,
#                                                            'renewable_energy':13, 'educational':14})

In [86]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10695 entries, 0 to 10694
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               10695 non-null  int64  
 1   Gender           10695 non-null  object 
 2   Ever_Married     10695 non-null  object 
 3   Age              10695 non-null  int64  
 4   Graduated        10695 non-null  object 
 5   Profession       10695 non-null  object 
 6   Work_Experience  10695 non-null  float64
 7   Spending_Score   10695 non-null  object 
 8   Family_Size      10695 non-null  float64
 9   Var_1            10695 non-null  object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 919.2+ KB


In [87]:
Dataset=pd.get_dummies(Dataset, columns=['Gender', 'Ever_Married','Graduated','Profession','Var_1','Spending_Score'], drop_first=True)

In [88]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10695 entries, 0 to 10694
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        10695 non-null  int64  
 1   Age                       10695 non-null  int64  
 2   Work_Experience           10695 non-null  float64
 3   Family_Size               10695 non-null  float64
 4   Segmentation              8068 non-null   object 
 5   Gender_Male               10695 non-null  uint8  
 6   Ever_Married_No           10695 non-null  uint8  
 7   Ever_Married_Yes          10695 non-null  uint8  
 8   Graduated_No              10695 non-null  uint8  
 9   Graduated_Yes             10695 non-null  uint8  
 10  Profession_Doctor         10695 non-null  uint8  
 11  Profession_Engineer       10695 non-null  uint8  
 12  Profession_Entertainment  10695 non-null  uint8  
 13  Profession_Executive      10695 non-null  uint8  
 14  Profes

In [89]:
Dataset.isnull().sum()
Dataset.shape

(10695, 28)

In [90]:
# Dataset['Account_ratio']=Dataset['Number_Open_Accounts']/Dataset['Total_Accounts']
# Dataset.head(5)

In [91]:
Dataset_test=Dataset[Dataset['Segmentation'].isnull()==True]
Dataset=Dataset[Dataset['Segmentation'].isnull()!=True]

In [92]:
Dataset_test.shape , Dataset.shape

((2627, 28), (8068, 28))

In [93]:
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8068 entries, 0 to 8067
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        8068 non-null   int64  
 1   Age                       8068 non-null   int64  
 2   Work_Experience           8068 non-null   float64
 3   Family_Size               8068 non-null   float64
 4   Segmentation              8068 non-null   object 
 5   Gender_Male               8068 non-null   uint8  
 6   Ever_Married_No           8068 non-null   uint8  
 7   Ever_Married_Yes          8068 non-null   uint8  
 8   Graduated_No              8068 non-null   uint8  
 9   Graduated_Yes             8068 non-null   uint8  
 10  Profession_Doctor         8068 non-null   uint8  
 11  Profession_Engineer       8068 non-null   uint8  
 12  Profession_Entertainment  8068 non-null   uint8  
 13  Profession_Executive      8068 non-null   uint8  
 14  Professi

In [94]:
X=Dataset.drop(['Segmentation', 'ID'], axis=1)
Y=Dataset['Segmentation']


X=X.iloc[:, :].values
Y=Y.iloc[:].values

# imputer=SimpleImputer(missing_values=np.nan, strategy='mean')

# X[:, 2:3]=imputer.fit_transform(X[:, 2:3])

# X[:, 6:7]=imputer.fit_transform(X[:, 6:7])


In [95]:
#rc=RandomForestClassifier(n_estimators=500, criterion='gini', max_depth=10, n_jobs=-1, random_state=100)
#rc.fit(X_train,Y_train)
#Y_predict=rc.predict(X_test)

#print(f1_score(Y_test,Y_predict, average='weighted'))


In [96]:
from sklearn.metrics import accuracy_score

In [97]:
from sklearn import tree

In [103]:
pd.DataFrame(X).isnull().sum()

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

# lg=LGBMClassifier(boosting_type='gbdt', max_depth=14, learning_rate=0.14, objective='multiclass', reg_alpha=0,
#                   reg_lambda=1, n_jobs=-1, random_state=100, n_estimators=600)

lg=LGBMClassifier()
#dt = tree.DecisionTreeClassifier()
lg.fit(X_train,Y_train)
Y_predict=lg.predict(X_test)

print(f1_score(Y_test,Y_predict, average='weighted'))
print(accuracy_score(Y_test,Y_predict))



0.5094396880772615
0.5173482032218092


In [99]:
X_main_test=Dataset_test.drop(['Segmentation', 'ID'], axis=1)

X_main_test=X_main_test.iloc[:, :].values

# X_main_test[:, 2:3]=imputer.fit_transform(X_main_test[:, 2:3])

# X_main_test[:, 6:7]=imputer.fit_transform(X_main_test[:, 6:7])

pd.DataFrame(X_main_test).isnull().sum()


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
dtype: int64

In [100]:
Y_test_predict=lg.predict(X_main_test)

sample_submission=pd.read_csv('../input/janatahack-customer-segmentation/sample_submission.csv')
sample_submission['Segmentation']=Y_test_predict

sample_submission.to_csv('basic_solution.csv', index=False)

# Yet to build the cross validation loop

In [101]:
X=pd.DataFrame(X)
Y=pd.DataFrame(Y)
X_main_test=pd.DataFrame(X_main_test)

kf=KFold(n_splits=10, random_state=100, shuffle=True)

y_test_predict=0
f1_score1 = 0
acc_score1 = 0
j=1
result={}
for i, (train_index, test_index) in enumerate(kf.split(Dataset)):
    
   Y_train, Y_valid = Y.iloc[train_index], Y.iloc[test_index]
   X_train, X_valid = X.iloc[train_index,:], X.iloc[test_index,:]
   
   print( "\nFold ", j)
    
   model=lg.fit(X_train,Y_train)
   pred = model.predict(X_valid)
   
   print(f1_score(Y_valid,pred, average='weighted'))
   print(accuracy_score(Y_valid,pred)) 
   f1_score1+=f1_score(Y_valid,pred, average='weighted')
   acc_score1+=accuracy_score(Y_valid,pred)
    
#    y_test_predict+=model.predict(X_main_test)  
   result[j]=model.predict(X_main_test)
   j+=1
   
    
# results=y_test_predict/10   
# results=np.round(results, decimals=0)
# print(f1_score1/10)

# d = pd.DataFrame()
# for i in range(1, 11):
#     d = pd.concat([d,pd.DataFrame(result[i])],axis=1)
# d.columns=['1','2','3','4','5','6','7','8','9','10']




Fold  1


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


0.5277908107276787
0.540272614622057

Fold  2
0.493039206748229
0.49566294919454773

Fold  3
0.5142754165425835
0.5229244114002478

Fold  4
0.5125403805043967
0.5179677819083024

Fold  5
0.5335871525214771
0.5415117719950434

Fold  6
0.5458751804474047
0.5514250309789344

Fold  7
0.5135379848797816
0.5241635687732342

Fold  8
0.5467595318523997
0.5551425030978935

Fold  9
0.5261758895985248
0.5310173697270472

Fold  10
0.5136120521273111
0.5210918114143921


In [102]:
re = d.mode(axis=1)[0]
sample_submission=pd.read_csv('C:/Users/UF145GV/Documents/DL/testing/Bank Statement/sample_submission_HSqiq1Q.csv')
sample_submission['Interest_Rate']=re

sample_submission.to_csv('C:/Users/UF145GV/Documents/DL/testing/Bank Statement/main_test.csv', index=False)

NameError: name 'd' is not defined

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns 
import plotly as py
import plotly.graph_objs as go
from sklearn.cluster import KMeans
import warnings
import os

In [ ]:
X1=Dataset.drop(['Segmentation', 'ID'], axis=1).iloc[: , :].values
# Y=Dataset['Segmentation']
# X1 = df[['Age' , 'Spending Score (1-100)']].iloc[: , :].values
inertia = []
for n in range(1 , 11):
    algorithm = (KMeans(n_clusters = n ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
    algorithm.fit(X1)
    inertia.append(algorithm.inertia_)

In [ ]:
plt.figure(1 , figsize = (15 ,6))
plt.plot(np.arange(1 , 11) , inertia , 'o')
plt.plot(np.arange(1 , 11) , inertia , '-' , alpha = 0.5)
plt.xlabel('Number of Clusters') , plt.ylabel('Inertia')
plt.show()

In [ ]:
algorithm = (KMeans(n_clusters = 4 ,init='k-means++', n_init = 10 ,max_iter=300, 
                        tol=0.0001,  random_state= 111  , algorithm='elkan') )
algorithm.fit(X1)
labels1 = algorithm.labels_
centroids1 = algorithm.cluster_centers_

In [ ]:
plt.figure(1 , figsize = (15 , 7) )
plt.clf()
Z = Z.reshape(xx.shape)
plt.imshow(Z , interpolation='nearest', 
           extent=(xx.min(), xx.max(), yy.min(), yy.max()),
           cmap = plt.cm.Pastel2, aspect = 'auto', origin='lower')

plt.scatter( x = 'Age' ,y = 'Spending Score (1-100)' , data = df , c = labels1 , 
            s = 200 )
plt.scatter(x = centroids1[: , 0] , y =  centroids1[: , 1] , s = 300 , c = 'red' , alpha = 0.5)
plt.ylabel('Spending Score (1-100)') , plt.xlabel('Age')
plt.show()